In [1]:
import pandas as pd

In [2]:
melon = pd.read_csv("../Data/melon_top2000.csv")

In [ ]:
melon

In [4]:
# !pip install selenium

In [8]:
!pip install webdriver-manager

  Using cached python_dotenv-0.20.0-py3-none-any.whl (17 kB)


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib.request as request
import time

In [6]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 1, 451, 50
# for문으로 인덱스 돌리기

total_lists = []
for page in list(range(1, 2000, 50)):
  
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
  driver.get('https://www.melon.com/genre/song_list.htm#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex='+ str(page))

  page_source = driver.page_source
  soup = BeautifulSoup(page_source, "html.parser")
  
  songIds = soup.select("#frm > div > table > tbody td:nth-child(4) > div > a")
  total_lists = total_lists + songIds

In [7]:
# songId 가져오기
songId_list = []
for songId in total_lists:
    songId_list.append(songId['href'].split('\'')[1])

len(songId_list)

2000

In [23]:
# 1000곡씩 나누기
songId_list_1000 =  songId_list[:1000]
songId_list_2000 =  songId_list[1000:]

In [22]:
len(songId_list[1000:])

1000

In [24]:
# 가수명, 노래제목, 발매일, 가사, 좋아요 수 리스트 변수 생성

singer_list = []
singName_list = []
date_list = []
lyrics_list = []
like_list = []
songImage_list =[]

# for문으로 가수명, 노래제목, 발매일, 가사, 좋아요 수 리스트 변수에 내용 담고 저장하기

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

for songId in songId_list_2000:
    
    driver.get(f'https://www.melon.com/song/detail.htm?songId={songId}')

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    try:
        time.sleep(2)
        # 가수명 가져오기
        artist_name = soup.select_one('#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1) ')
        singer_list.append(artist_name.string)

        # 노래제목 가져오기
        song_name = soup.select_one("#downloadfrm > div > div > div.entry > div.info > div.song_name ")
        singName_list.append(song_name.text.strip().split('\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t')[1])

        # 발매일 가져오기
        date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)')
        date_list.append(date.string)

        # 가사 가져오기
        lyric1 = str(soup.select_one('#d_video_summary')).replace('<br/>',' ')\
                                                         .replace('<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->',' ')\
                                                         .replace('</div>',' ').replace('\t','').replace('\n',' ')
        lyric = lyric1.replace('  ', ' ')
        lyrics_list.append(lyric)

        # 좋아요수 가져오기
        like = soup.select_one("#d_like_count")
        like_list.append(like.text)

        # 이미지 링크 가져오기
        # 앨범 사진 가져오기
        song_image = soup.select_one('#downloadfrm > div > div > div.thumb > a.image_typeAll > img')["src"]
        songImage_list.append(song_image)
        time.sleep(2)
    except:
        singer_list.append(0)
        singName_list.append(0)
        date_list.append(0)
        lyrics_list.append(0)
        like_list.append(0)
        songImage_list.append(0)
        time.sleep(1)

In [28]:
print(len(singer_list))
print(len(singName_list))
print(len(date_list))
print(len(lyrics_list))
print(len(like_list))
print(len(songImage_list))

1000
1000
1000
1000
1000
1000


In [36]:
ballad = {'가수명':singer_list, '노래제목':singName_list, '발매일':date_list, '가사':lyrics_list, '좋아요수':like_list, '앨범사진':songImage_list}

In [37]:
import pandas as pd

In [38]:
ballad_1001_2000 =  pd.DataFrame(ballad)

In [46]:
ballad_1001_2000.tail()

,가수명,노래제목,발매일,가사,좋아요수,앨범사진
995,어반자카파,똑같은 사랑 똑같은 이별,2012.10.30,욕심에 점점 초라해지는 붙잡으려 할수록 더 멀어지는 지키려 할수록 지쳐만 가는 그...,"56,338",https://cdnimg.melon.co.kr/cm/album/images/021...
996,잔나비,소란한 밤을 지나,2021.03.24,말하지마 지금 이 거리가 좋아 그대로 머물러줘 비스듬한 채로 그 말 뒤에 이름만 ...,"11,883",https://cdnimg.melon.co.kr/cm2/album/images/10...
997,M.Street (엠스트리트),For My Love,2004.01.06,그대 이름 부르며 희미하게 웃어보죠 나눠 가진 추억이 그대 맘을 대신하네요 긴 ...,"14,059",https://cdnimg.melon.co.kr/cm/album/images/000...
998,씨야,Promise U,2006.02.22,한동안 많이 힘들겠지 예전처럼 우리다시 만날까 말하고 싶은데 말하고 싶은데 그댄 ...,"5,408",https://cdnimg.melon.co.kr/cm/album/images/003...
999,이홍기 (FT아일랜드),서쪽하늘 (여심도둑 괴도루팡),2017.04.09,서쪽 하늘로 노을은 지고 이젠 슬픔이 돼 버린 그대를 다시 부를 수 없을 것 같아...,"5,435",https://cdnimg.melon.co.kr/cm/album/images/100...


In [44]:
len(ballad_1001_2000['노래제목'].unique())

965

In [45]:
ballad_1001_2000.isnull().sum()

가수명     0
노래제목    0
발매일     0
가사      0
좋아요수    0
앨범사진    0
dtype: int64